# ![HELP LOGO](https://avatars1.githubusercontent.com/u/7880370?s=100&v=4>)SEIP MIPS map homogenisation


The original MIPS maps have been produced by IPAC. This code takes the individual image files and adds them into a single multi-layered fits files to have the homogeneous HELP format.  

## Caveats 
* Although many of the longer wavelength MIPS maps are available I've only run this on the MIPS 24micron maps at the moment
* Some of the SEIP maps may not have successfully down-loaded, this hasn't been checked
* only puts in image, error and coverage files (and adds blank PSF extension)

<h2> Guide</h2>
<h3>Steps: </h3>
* Obtain list of fields from sub-folders
for each field:
* read in coverage, image and noise
* attach meta data
* create PSF extension
* Save out

<h3>Things you may wish to edit</h3>
* You'll need to change the folder path in the 2nd code block to point you where the files are in your system
* Add in a specifc field (or switch to wild chracter) in the path to pick field(s)
* change the 1.mosaic pattern to 2.mosaic for MIPS 70 micron or 3.mosaic for MIPS 160.
* read in more of the available files and add as additional extensions

# Importing libraries

In [2]:
from datetime import datetime
from itertools import product
import glob

import numpy as np
import astropy as astropy

from astropy.io import fits
from astropy.table import Table
from astropy.table import Column
from astropy import wcs
VERSION = "0.9"

# This is where you set up the field and folders to work on
the notebook needs to be run for each field in turn - commenting in/out as appropriate

In [2]:
#field_specified='HDF-N' 
#field_specified='XMM-LSS' 
#field_specified='AKARI-NEP' 
#field_specified='AKARI-SEP' 
#field_specified='Bootes' 
#field_specified='CDFS-SWIRE' 
#field_specified='COSMOS' 
#field_specified='EGS' 
#field_specified='ELAIS-N1' 
#field_specified='ELAIS-N2' 
#field_specified='ELAIS-S1' 
#field_specified='GAMA-09' 
#field_specified='GAMA-12' 
#field_specified='GAMA-15' 
#field_specified='NGP' 
#field_specified='SGP' 
#field_specified='Lockman-SWIRE' 
#field_specified='SA13' 
#field_specified='SPIRE-NEP' 
#field_specified='SSDF' 
#field_specified='xFLS' 
#field_specified='XMM-13hr' 
#field_specified='Herschel-Stripe-82' 


#mainfolder='/Volumes/Seagate Backup Plus Drive/HELP/mips_fields'
#moc_dir='/Volumes/Seagate Backup Plus Drive/HELP/dmu_products/dmu2/dmu2_field_coverages/'

#mainfolder='/Users/sjo/Data/SEIP/mips_fields'
#moc_dir='/Users/sjo/Data/dmu2_field_coverages/'

mainfolder='../../dmu17/dmu17_SEIP_maps/'
moc_dir=   '../../dmu2/dmu2_field_coverages/'

# finding all the SEIP mosaic files in the folder

In [3]:
# finding all the SEIP mosaic files (other files should be there if the mosaic file is)
filelist= glob.glob(f"{mainfolder}/{field_specified}/data/*1.mosaic.fits", recursive=True)
print("Number of files found:", len(filelist))

Number of files found: 466


# opening the table which logs the SEIP files

In [4]:
# opening the table of SEIP in which we will update some records
seip_table = fits.open(f"{mainfolder}/{field_specified}/mosaic_tiles_{field_specified}.fits", mode='update')
seip_table.info()
seip_hd=seip_table[1].header
seip_data=seip_table[1].data
#print(repr(seip_hd))
seip_table2=Table.read(f"{mainfolder}/{field_specified}/mosaic_tiles_{field_specified}.fits")
seip_table2['areahelp']=np.zeros(len(seip_table2['EXPTIME']))
#t.add_column(aa, index=0) 


Filename: ../../dmu17/dmu17_SEIP_maps//Herschel-Stripe-82/mosaic_tiles_Herschel-Stripe-82.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1  mosaic_tiles.cat    1 BinTableHDU    104   2161R x 37C   [J, 20A, 4A, I, J, D, D, D, D, D, D, D, I, J, I, I, I, I, 22A, 29A, D, D, D, D, D, D, D, D, D, D, E, D, 18A, L, 143A, 44A, L]   


# MOC libraries
This next block of code comes from Peter Hurley's XID libraray and allows us to create a MOC from the error array

In [5]:
import pymoc as pymoc
from pymoc import MOC
from pymoc import io

from healpy import pixelfunc
from pymoc.util import catalog

import numpy as np
def get_HEALPix_pixels(order,ra,dec,unique=True):


    """Work out what HEALPix a source is in
    :param order: order of HEALPix
    :param ra: Right Ascension
    :param dec: Declination
    :param unique: if unique is true, removes duplicate pixels
    :return: list of HEALPix pixels :rtype:
    """
    HPX_D2R=np.pi/180.0
    #Convert catalogue to polar co-ords in radians
    phi = ra*HPX_D2R
    theta = np.pi/2.0 - dec*HPX_D2R
    #calculate what pixel each object is in
    ipix = pixelfunc.ang2pix(2**order, theta, phi, nest=True)
    #return unique pixels (i.e. remove duplicates)
    if unique is True:
        return np.unique(ipix)
    else:
        return ipix


def create_MOC_from_map(good,wcs):
    """Generate MOC from map
    :param good: boolean array associated with map
    :param wcs: wcs information
    :return: MOC :rtype: pymoc.MOC
    """
    x_pix,y_pix=np.meshgrid(np.arange(0,wcs._naxis1),np.arange(0,wcs._naxis2))
    ra,dec= wcs.wcs_pix2world(x_pix,y_pix,0)

    pixels=get_HEALPix_pixels(15,ra[good],dec[good])
    map_moc=MOC()
    map_moc.add(15,pixels)
    return map_moc

# setting up global MOCS

In [6]:
# sum of all SEIP MOCS
total_moc=MOC()

# Performing the homogenisation
i.e. merging component files into a single multi-extension file and adding some keywords
traps errors so that it can continue, e.g. if a file is missing
skips over files if they've already been logged as "downloaded" (this could be improved to be an option

In [7]:
for i, file in enumerate(filelist):
    print('----------------')
    print(i,file)
    folders=file.split('/')
    print(folders[len(folders)-3])
    field=folders[len(folders)-3]
    filename=folders[len(folders)-1]
    seip_id=(filename==seip_data.field('file')) 
    print(filename,seip_data.field('file')[seip_id])
# actual HELP SPIRE MOC
    print(f'{moc_dir}{field}_MOC.fits')
#    help_moc_hdu = fits.open(f'{moc_dir}{field}_MOC.fits')
    help_moc=MOC()
#    pymoc.io.fits.read_moc_fits(help_moc, f'{moc_dir}{field}_MOC.fits',include_meta=False)
    help_moc.read(f'{moc_dir}{field}_MOC.fits')

    if seip_data.field('downloaded')[seip_id]:
        print("Already downloaded skipping file",filename)
    else:
        ext=0
    
        try:

            # Load in image map
            image_map=fits.open(file)
            image_hdu = fits.ImageHDU(header=image_map[ext].header,
                                data=image_map[ext].data)
            image_hdu.header['EXTNAME'] = "IMAGE"
 
            # Load in uncertainty map
            noise_map=fits.open(file.replace('mosaic','unc'))
            noise_hdu = fits.ImageHDU(header=noise_map[ext].header,
                                data=noise_map[ext].data)
            noise_hdu.header['EXTNAME'] = "ERROR"
 
            # we'll use the uncertainty map to generate a MOC
            good=~np.isnan(noise_map[ext].data)
            w = wcs.WCS(noise_map[ext].header)     
            m=create_MOC_from_map(good,w)
            total_moc=total_moc+m
        
            intersect=m.intersection(help_moc)
            print('Area',repr(intersect.area))
        
#        moc_hdu=io.fits.write_moc_fits_hdu(m)
#        io.fits.write_moc_fits(m,file.replace('mosaic','moc3'),overwrite=True)
        
            m.write(file.replace('mosaic','moc'), overwrite=True)
        
            # Load in coverage map
            cov_map=fits.open(file.replace('mosaic','cov'))
            cov_hdu = fits.ImageHDU(header=cov_map[ext].header,
                                data=cov_map[ext].data)
            cov_hdu.header['EXTNAME'] = "COVERAGE"

            # Mark this tile as successfully downloaded
        
            seip_data.field('downloaded')[seip_id]=True
            print('downloaded:',seip_data.field('downloaded')[seip_id])

# ----------------------------
            # mark this tile with area of overlap with HELP
            seip_data.field('inhelp')[seip_id]=(intersect.area>0.)
            seip_table2.field('areahelp')[seip_id]=(intersect.area)
            print('updating intersection with HELP')
            #print(seip_id) 
# ----------------------------

        except:
            # update the SEIP file to clearly indicate that this file is not there
            seip_data.field('downloaded')[seip_id]=False
            print,'error'
#            stop
    #fits.close(file)
        pass
            
            #Create primary header
        primary_hdu = fits.PrimaryHDU()
        primary_hdu.header.append((
            "CREATOR", "Herschel Extragalactic Legacy Project"
        ))
        primary_hdu.header.append((
            "TIMESYS", "UTC", "All dates are in UTC time"
        ))
        primary_hdu.header.append((
            "DATE", datetime.now().replace(microsecond=0).isoformat(),
            "Date of file creation"
        ))
        primary_hdu.header.append((
        "VERSION", VERSION, "HELP product version"
        ))
        primary_hdu.header.append((
        "TELESCOP", "Spitzer", "Name of the telescope"
        ))
        primary_hdu.header.append((
        "INSTRUME", "MIPS", "Name of the instrument"
        ))
        primary_hdu.header.append((
        "FILTER", "MIPS-24", "Name of the filter"
        ))
        primary_hdu.header.append((
        "FIELD", field, "Name of the HELP field"
        ))


        #Add empty extension for PSF
        psf_hdu = fits.ImageHDU()
        psf_hdu.header['EXTNAME'] = "PSF"
        psf_hdu.header.add_comment("The PSF is not available.")

        hdu_list = fits.HDUList([primary_hdu, image_hdu, noise_hdu, cov_hdu, psf_hdu])
        hdu_list.writeto(file.replace('mosaic','help'),
                 checksum=True, overwrite=True)
       # print(f"{file} /  processed...")

----------------
0 ../../dmu17/dmu17_SEIP_maps//Herschel-Stripe-82/data/20000861.20000861-140.MIPS.1.mosaic.fits
Herschel-Stripe-82
20000861.20000861-140.MIPS.1.mosaic.fits ['20000861.20000861-140.MIPS.1.mosaic.fits']
../../dmu2/dmu2_field_coverages/Herschel-Stripe-82_MOC.fits
Area 1.4663315547854497e-05
downloaded: [ True]
updating intersection with HELP
----------------
1 ../../dmu17/dmu17_SEIP_maps//Herschel-Stripe-82/data/20000821.20000821-576.MIPS.1.mosaic.fits
Herschel-Stripe-82
20000821.20000821-576.MIPS.1.mosaic.fits ['20000821.20000821-576.MIPS.1.mosaic.fits']
../../dmu2/dmu2_field_coverages/Herschel-Stripe-82_MOC.fits
Area 0.0
downloaded: [ True]
updating intersection with HELP
----------------
2 ../../dmu17/dmu17_SEIP_maps//Herschel-Stripe-82/data/20000861.20000861-627.MIPS.1.mosaic.fits
Herschel-Stripe-82
20000861.20000861-627.MIPS.1.mosaic.fits ['20000861.20000861-627.MIPS.1.mosaic.fits']
../../dmu2/dmu2_field_coverages/Herschel-Stripe-82_MOC.fits
Area 0.0
downloaded: [ Tr

Area 7.327269019689447e-05
downloaded: [ True]
updating intersection with HELP
----------------
24 ../../dmu17/dmu17_SEIP_maps//Herschel-Stripe-82/data/20001201.20001201-58.MIPS.1.mosaic.fits
Herschel-Stripe-82
20001201.20001201-58.MIPS.1.mosaic.fits ['20001201.20001201-58.MIPS.1.mosaic.fits']
../../dmu2/dmu2_field_coverages/Herschel-Stripe-82_MOC.fits
Area 4.492133782069692e-06
downloaded: [ True]
updating intersection with HELP
----------------
25 ../../dmu17/dmu17_SEIP_maps//Herschel-Stripe-82/data/20000821.20000821-173.MIPS.1.mosaic.fits
Herschel-Stripe-82
20000821.20000821-173.MIPS.1.mosaic.fits ['20000821.20000821-173.MIPS.1.mosaic.fits']
../../dmu2/dmu2_field_coverages/Herschel-Stripe-82_MOC.fits
Area 6.966513421342164e-05
downloaded: [ True]
updating intersection with HELP
----------------
26 ../../dmu17/dmu17_SEIP_maps//Herschel-Stripe-82/data/20000791.20000791-628.MIPS.1.mosaic.fits
Herschel-Stripe-82
20000791.20000791-628.MIPS.1.mosaic.fits ['20000791.20000791-628.MIPS.1.mos

Area 2.0359918548520814e-05
downloaded: [ True]
updating intersection with HELP
----------------
48 ../../dmu17/dmu17_SEIP_maps//Herschel-Stripe-82/data/20000021.20000021-0.MIPS.1.mosaic.fits
Herschel-Stripe-82
20000021.20000021-0.MIPS.1.mosaic.fits ['20000021.20000021-0.MIPS.1.mosaic.fits']
../../dmu2/dmu2_field_coverages/Herschel-Stripe-82_MOC.fits
Area 2.8334772637965858e-05
downloaded: [ True]
updating intersection with HELP
----------------
49 ../../dmu17/dmu17_SEIP_maps//Herschel-Stripe-82/data/20000141.20000141-2.MIPS.1.mosaic.fits
Herschel-Stripe-82
20000141.20000141-2.MIPS.1.mosaic.fits ['20000141.20000141-2.MIPS.1.mosaic.fits']
../../dmu2/dmu2_field_coverages/Herschel-Stripe-82_MOC.fits
Area 7.297035379384591e-06
downloaded: [ True]
updating intersection with HELP
----------------
50 ../../dmu17/dmu17_SEIP_maps//Herschel-Stripe-82/data/20000791.20000791-383.MIPS.1.mosaic.fits
Herschel-Stripe-82
20000791.20000791-383.MIPS.1.mosaic.fits ['20000791.20000791-383.MIPS.1.mosaic.fit

Area 2.120938631321528e-05
downloaded: [ True]
updating intersection with HELP
----------------
72 ../../dmu17/dmu17_SEIP_maps//Herschel-Stripe-82/data/20001541.20001541-0.MIPS.1.mosaic.fits
Herschel-Stripe-82
20001541.20001541-0.MIPS.1.mosaic.fits ['20001541.20001541-0.MIPS.1.mosaic.fits']
../../dmu2/dmu2_field_coverages/Herschel-Stripe-82_MOC.fits
Area 1.3756306338708859e-05
downloaded: [ True]
updating intersection with HELP
----------------
73 ../../dmu17/dmu17_SEIP_maps//Herschel-Stripe-82/data/20000821.20000821-593.MIPS.1.mosaic.fits
Herschel-Stripe-82
20000821.20000821-593.MIPS.1.mosaic.fits ['20000821.20000821-593.MIPS.1.mosaic.fits']
../../dmu2/dmu2_field_coverages/Herschel-Stripe-82_MOC.fits
Area 0.0
downloaded: [ True]
updating intersection with HELP
----------------
74 ../../dmu17/dmu17_SEIP_maps//Herschel-Stripe-82/data/20001141.20001141-19.MIPS.1.mosaic.fits
Herschel-Stripe-82
20001141.20001141-19.MIPS.1.mosaic.fits ['20001141.20001141-19.MIPS.1.mosaic.fits']
../../dmu2/d

Area 7.375447788433312e-05
downloaded: [ True]
updating intersection with HELP
----------------
96 ../../dmu17/dmu17_SEIP_maps//Herschel-Stripe-82/data/20000801.20000801-593.MIPS.1.mosaic.fits
Herschel-Stripe-82
20000801.20000801-593.MIPS.1.mosaic.fits ['20000801.20000801-593.MIPS.1.mosaic.fits']
../../dmu2/dmu2_field_coverages/Herschel-Stripe-82_MOC.fits
Area 0.0
downloaded: [ True]
updating intersection with HELP
----------------
97 ../../dmu17/dmu17_SEIP_maps//Herschel-Stripe-82/data/20000801.20000801-558.MIPS.1.mosaic.fits
Herschel-Stripe-82
20000801.20000801-558.MIPS.1.mosaic.fits ['20000801.20000801-558.MIPS.1.mosaic.fits']
../../dmu2/dmu2_field_coverages/Herschel-Stripe-82_MOC.fits
Area 0.0
downloaded: [ True]
updating intersection with HELP
----------------
98 ../../dmu17/dmu17_SEIP_maps//Herschel-Stripe-82/data/20000821.20000821-123.MIPS.1.mosaic.fits
Herschel-Stripe-82
20000821.20000821-123.MIPS.1.mosaic.fits ['20000821.20000821-123.MIPS.1.mosaic.fits']
../../dmu2/dmu2_field_

...

Area 1.7163930184681828e-05
downloaded: [ True]
updating intersection with HELP
----------------
360 ../../dmu17/dmu17_SEIP_maps//Herschel-Stripe-82/data/40007680.40007680-0.MIPS.1.mosaic.fits
Herschel-Stripe-82
40007680.40007680-0.MIPS.1.mosaic.fits ['40007680.40007680-0.MIPS.1.mosaic.fits']
../../dmu2/dmu2_field_coverages/Herschel-Stripe-82_MOC.fits
Area 1.3179916615477598e-05
downloaded: [ True]
updating intersection with HELP
----------------
361 ../../dmu17/dmu17_SEIP_maps//Herschel-Stripe-82/data/20000801.20000801-645.MIPS.1.mosaic.fits
Herschel-Stripe-82
20000801.20000801-645.MIPS.1.mosaic.fits ['20000801.20000801-645.MIPS.1.mosaic.fits']
../../dmu2/dmu2_field_coverages/Herschel-Stripe-82_MOC.fits
Area 0.0
downloaded: [ True]
updating intersection with HELP
----------------
362 ../../dmu17/dmu17_SEIP_maps//Herschel-Stripe-82/data/20000801.20000801-0.MIPS.1.mosaic.fits
Herschel-Stripe-82
20000801.20000801-0.MIPS.1.mosaic.fits ['20000801.20000801-0.MIPS.1.mosaic.fits']
../../dmu2/

Area 0.0
downloaded: [ True]
updating intersection with HELP
----------------
384 ../../dmu17/dmu17_SEIP_maps//Herschel-Stripe-82/data/20000801.20000801-471.MIPS.1.mosaic.fits
Herschel-Stripe-82
20000801.20000801-471.MIPS.1.mosaic.fits ['20000801.20000801-471.MIPS.1.mosaic.fits']
../../dmu2/dmu2_field_coverages/Herschel-Stripe-82_MOC.fits
Area 1.2811261259502271e-05
downloaded: [ True]
updating intersection with HELP
----------------
385 ../../dmu17/dmu17_SEIP_maps//Herschel-Stripe-82/data/20000850.20000850-0.MIPS.1.mosaic.fits
Herschel-Stripe-82
20000850.20000850-0.MIPS.1.mosaic.fits ['20000850.20000850-0.MIPS.1.mosaic.fits']
../../dmu2/dmu2_field_coverages/Herschel-Stripe-82_MOC.fits
Area 4.160246433819948e-05
downloaded: [ True]
updating intersection with HELP
----------------
386 ../../dmu17/dmu17_SEIP_maps//Herschel-Stripe-82/data/20000861.20000861-628.MIPS.1.mosaic.fits
Herschel-Stripe-82
20000861.20000861-628.MIPS.1.mosaic.fits ['20000861.20000861-628.MIPS.1.mosaic.fits']
../../

Area 2.8149469681258686e-05
downloaded: [ True]
updating intersection with HELP
----------------
408 ../../dmu17/dmu17_SEIP_maps//Herschel-Stripe-82/data/20001281.20001281-51.MIPS.1.mosaic.fits
Herschel-Stripe-82
20001281.20001281-51.MIPS.1.mosaic.fits ['20001281.20001281-51.MIPS.1.mosaic.fits']
../../dmu2/dmu2_field_coverages/Herschel-Stripe-82_MOC.fits
Area 0.0
downloaded: [ True]
updating intersection with HELP
----------------
409 ../../dmu17/dmu17_SEIP_maps//Herschel-Stripe-82/data/20000821.20000821-122.MIPS.1.mosaic.fits
Herschel-Stripe-82
20000821.20000821-122.MIPS.1.mosaic.fits ['20000821.20000821-122.MIPS.1.mosaic.fits']
../../dmu2/dmu2_field_coverages/Herschel-Stripe-82_MOC.fits
Area 5.601513325513954e-05
downloaded: [ True]
updating intersection with HELP
----------------
410 ../../dmu17/dmu17_SEIP_maps//Herschel-Stripe-82/data/20000801.20000801-592.MIPS.1.mosaic.fits
Herschel-Stripe-82
20000801.20000801-592.MIPS.1.mosaic.fits ['20000801.20000801-592.MIPS.1.mosaic.fits']
../

Area 2.033456130181352e-05
downloaded: [ True]
updating intersection with HELP
----------------
432 ../../dmu17/dmu17_SEIP_maps//Herschel-Stripe-82/data/20000861.20000861-645.MIPS.1.mosaic.fits
Herschel-Stripe-82
20000861.20000861-645.MIPS.1.mosaic.fits ['20000861.20000861-645.MIPS.1.mosaic.fits']
../../dmu2/dmu2_field_coverages/Herschel-Stripe-82_MOC.fits
Area 0.0
downloaded: [ True]
updating intersection with HELP
----------------
433 ../../dmu17/dmu17_SEIP_maps//Herschel-Stripe-82/data/20001321.20001321-58.MIPS.1.mosaic.fits
Herschel-Stripe-82
20001321.20001321-58.MIPS.1.mosaic.fits ['20001321.20001321-58.MIPS.1.mosaic.fits']
../../dmu2/dmu2_field_coverages/Herschel-Stripe-82_MOC.fits
Area 2.036967133571593e-05
downloaded: [ True]
updating intersection with HELP
----------------
434 ../../dmu17/dmu17_SEIP_maps//Herschel-Stripe-82/data/20000791.20000791-314.MIPS.1.mosaic.fits
Herschel-Stripe-82
20000791.20000791-314.MIPS.1.mosaic.fits ['20000791.20000791-314.MIPS.1.mosaic.fits']
../.

Area 2.8243096438331784e-05
downloaded: [ True]
updating intersection with HELP
----------------
456 ../../dmu17/dmu17_SEIP_maps//Herschel-Stripe-82/data/20000271.20000271-0.MIPS.1.mosaic.fits
Herschel-Stripe-82
20000271.20000271-0.MIPS.1.mosaic.fits ['20000271.20000271-0.MIPS.1.mosaic.fits']
../../dmu2/dmu2_field_coverages/Herschel-Stripe-82_MOC.fits
Area 5.578496747733483e-05
downloaded: [ True]
updating intersection with HELP
----------------
457 ../../dmu17/dmu17_SEIP_maps//Herschel-Stripe-82/data/20000821.20000821-401.MIPS.1.mosaic.fits
Herschel-Stripe-82
20000821.20000821-401.MIPS.1.mosaic.fits ['20000821.20000821-401.MIPS.1.mosaic.fits']
../../dmu2/dmu2_field_coverages/Herschel-Stripe-82_MOC.fits
Area 2.2728870558214102e-05
downloaded: [ True]
updating intersection with HELP
----------------
458 ../../dmu17/dmu17_SEIP_maps//Herschel-Stripe-82/data/20000821.20000821-366.MIPS.1.mosaic.fits
Herschel-Stripe-82
20000821.20000821-366.MIPS.1.mosaic.fits ['20000821.20000821-366.MIPS.1.m

# Wrapping up some output files

In [8]:
# writing out the SEIP Table
seip_table.flush()
seip_table2.write(f"{mainfolder}/{field_specified}/mosaic_tiles_{field_specified}_new.fits", overwrite=True)

# writing out the combined moc
total_moc.write('help_moc.fits', overwrite=True)

<h1> Finding MOC files and writing an Aladin Script for plotting then up </h1>

In [9]:
# writing a file for plotting up MOCS in aladin

moc_filelist= glob.glob(f"{mainfolder}/{field_specified}/data/*1.moc.fits", recursive=True)
print("Number of MOC files found:", len(moc_filelist))


f=open(f"{mainfolder}/{field_specified}/aladin_script_{field_specified}.txt", 'w')
f.write("# This script can be loaded into Aladin using the Tools>Script Console (F5)\n")
f.write("# you will need to alter the file paths to be correct relative to your Aladin installation\n")
f.write("load "+moc_dir+field+"_MOC.fits \n")
#    help_moc_hdu = fits.open(f'{moc_dir}{field}_MOC.fits')

for i, file in enumerate(moc_filelist):
    f.write("load "+file.replace('mosaic','moc')+"\n")
f.write("save "+f"{mainfolder}/{field_specified}/{field_specified}_moc_plots.eps")
f.close()
        

Number of MOC files found: 466



-------------------------------------------------------------------------------

![HELP LOGO](https://avatars1.githubusercontent.com/u/7880370?s=75&v=4)

**Authors**:  This code was adapted from code used for SPIRE homogenisaton (Yannick Roehlly) and in turn adapted for PACS data (Peter Hurley). [Seb Oliver](http://www.sussex.ac.uk/profiles/91548)

 
For a full description of the database and how it is organised in to `dmu_products` please the top level [readme](../readme.md).
 
The Herschel Extragalactic Legacy Project, ([HELP](http://herschel.sussex.ac.uk/)), is a [European Commission Research Executive Agency](https://ec.europa.eu/info/departments/research-executive-agency_en)
funded project under the SP1-Cooperation, Collaborative project, Small or medium-scale focused research project, FP7-SPACE-2013-1 scheme, Grant Agreement
Number 607254.

[Acknowledgements](http://herschel.sussex.ac.uk/acknowledgements)

